In [ ]:
!apt-get install -qq libgdal-dev libproj-dev
!pip install --no-binary shapely shapely --force
!pip install cartopy
!pip install regionmask


#!pip install regionmask
#!pip install cartopy

#!pip install -q --upgrade ipython
#!pip install -q --upgrade ipykernel

In [ ]:
#import required packages
import os
import warnings
import time
import regionmask
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import geopandas as gpd
import cartopy.crs as ccrs
import shapely
from datetime import datetime as dt
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from scipy.interpolate import interp1d
from dateutil.relativedelta import relativedelta
from google.colab import drive
from google.colab import files

In [ ]:
#establish working directory and mount drive
drive.mount('/content/drive')
working_directory = '/content/drive/My Drive/COS Seesaw Research'

Parallel site arrays for constructing regions, and time delta for building feature time series in addition, contains the radius for defining map regions

In [ ]:
cos_sites = ['alt', 'brw', 'cgo', 'hfm', 'kum', 'lef', 'mhd', 'mlo', 'nwr', 'psa', 'smo', 'spo', 'sum', 'thd']
cos_site_centers = [(-62.3, 82.5), (-156.6, 71.3), (144.7,-40.7), (-72.2, 42.5), (-154.8, 19.5), (-90.3, 45.6), (-9.9, 53.3), (-155.6, 19.5), (-105.5, 40.1), (-64.0, -64.6), (-170.6, -14.2), (0, -90), (-38.4, 72.6), (-124.1,41.0)]
time_delta_general = [('-15d', relativedelta(days=-15)), ('-1m', relativedelta(months=-1)), ('-1m15d', relativedelta(months=-1, days=-15)), ('-2m', relativedelta(months=-2))]
region_size = 30
year_start = 2000
year_end = 2018
divider = ('-------------------------------------------------------------------------------------------------------')

Build regions

In [ ]:
regions = None
names = []
abbrevs = []
region_list = []

for i in range(len(cos_sites)):
  names.append(cos_sites[i])
  abbrevs.append(cos_sites[i])
  center_point = Point(cos_site_centers[i][0], cos_site_centers[i][1])
  circle = center_point.buffer(region_size)

  #region_bound = np.array([list(cos_site_centers[i])])
  
  #region_list.append(region_bound)
  region_list.append(circle)

regions = regionmask.Regions(region_list, names=names, abbrevs=abbrevs, name='Ocean Regions', overlap=True)
plt.figure(figsize=(24,12))
regions.plot(label='abbrev')
plt.show()

Establish dictionary of different data frames, one for each of the observation sites

In [ ]:
days = pd.date_range(start=dt(year=2005, month=1, day=1), end=dt(year=2017, month=1, day=1), freq='SM')
days_frame = pd.DataFrame({'time':days})

Helper function for getting regionalized data

In [ ]:
def standardize_longitude(data):
    print(divider)
    print('Standardizing Longitude')
    print(divider)
    data = data.assign_coords(lon = (((data.lon + 180) % 360) -180))
    data = data.sortby(data.lon)
    print('Done standardizing longitude')
    print(divider)
    return data

In [ ]:
def standardize_time_Month(data, year, tName_start, tName_end):
    print(divider)
    print('Standardizing time')
    print(divider)
    dates = []
    for val in data[tName_start].values:
        date = dt(year=year, month=int(val), day=1)
        dates.append(date)
    data = data.rename({tName_start : tName_end})
    data = data.assign_coords(time=dates)
    print('Done standardizing time')
    print(divider)
    return data

In [ ]:
def get_regionalized_map_data(file_name_start, file_name_end, variable_name, regions, start, end, standardizeLon=True, standardizeTime=None):
  data_dict = {}
  for region in regions:
    data_dict[region.abbrev] = []
  
  for year in range(start, end + 1):
    f_name = file_name_start + str(year) + file_name_end
    data = None
    # Some of the downward solar radiation data used a date time format that
    # xarray did not like, this attempts to manually convert it, but may need to be
    # adjusted for future issues
    try:
        data = xr.open_dataset(f_name).load()
    except:
        print("Error opening " + f_name)
        print("Attempting alternative datetime conversion")
        try:
            data = xr.open_dataset(f_name, decode_times=False).load()
            time_since = dt(year=1800, month=1, day=1)
            temp_time = data['time'].to_series()
            funct = lambda x : relativedelta(hours=x)
            time_col = temp_time.apply(funct)
            funct2 = lambda x: time_since + x
            data['time'] = time_col.apply(funct2)
            print("Success!")
        except Exception as e:
            print("Failed")
            print(e)
            exit()
          
    # If necessary, convert 0 to 360 to -180 to 180
    if standardizeLon:
        data = standardize_longitude(data)

    if standardizeTime is not None:
        data = standardize_time_Month(data, year, standardizeTime[0], standardizeTime[1])

    # get the mean for each region
    #data_mask = regions.mask(data)
    data_mask = regions.mask_3D(data)
    for region in regions:
        region_index = regions.map_keys(region.name)
        region_mask = data_mask.sel(region=region_index)
        region_data = data.where(region_mask)
        data_mean = region_data.mean(dim=('lat', 'lon'))
        data_dict[region.abbrev].append(data_mean)

    print('Region Data added')
    data.close()
    gc.collect()


  # combine each region's yearly mean into single data set
  for region in data_dict.keys():
      data_dict[region] = xr.concat(data_dict[region], dim='time')

  return data_dict
  

Helper function for interpolating using cubic

In [ ]:
def interp_data(region_var, interp_dates):
  interp = region_var.interp(time=list(map(str, interp_dates)), method='cubic')
  return interp

Add features to region dataframes

In [ ]:
offset_dates = []
for delta in time_delta_general:
  funct = lambda x: x + delta[1]
  current_offset = days_frame['time'].apply(funct)
  offset_dates.append((delta[0], current_offset))

Add SST -- disabled to see if we can survive mask 3d on data with different granularity

In [ ]:
# add sea surface temperature data

f_name = working_directory + '/Data/sst/sst.day.mean.'
sst_dict = get_regionalized_map_data(f_name, '.nc', '_sst', regions, year_start, year_end)

for region in sst_dict.keys():
  interp = interp_data(sst_dict[region].sst, days_frame['time'])
  column_name = region + '_sst'
  days_frame[column_name] = interp

  for delta in offset_dates:
    column_name = region + '_sst' + delta[0]
    interp = interp_data(sst_dict[region].sst, delta[1])
    days_frame[column_name] = interp
  
days_frame = days_frame.copy()


Add CDOM

In [ ]:
f_name = working_directory + '/Data/CDOM/CDOM_a350_'
cdom_dict = get_regionalized_map_data(f_name, '.nc', '_cdom', regions, year_start, year_end, standardizeTime=('month', 'time'))
for region in cdom_dict.keys():
  interp = interp_data(cdom_dict[region].CDOM_a350, days_frame['time'])
  column_name = region + '_cdom'
  days_frame[column_name] = interp

  for delta in offset_dates:
    column_name = region + '_cdom' + delta[0]
    interp = interp_data(cdom_dict[region].CDOM_a350, delta[1])
    days_frame[column_name] = interp
      
days_frame = days_frame.copy()

Add downward radiation

In [ ]:
f_name = working_directory + '/Data/DSRF/dswrf.sfc.gauss.'
dswrf_dict = get_regionalized_map_data(f_name, '.nc', 'dswrf', regions, year_start, year_end)

for region in dswrf_dict.keys():
  interp = interp_data(dswrf_dict[region].dswrf, days_frame['time'])
  column_name = region + '_dswrf'
  days_frame[column_name] = interp

  for delta in offset_dates:
    column_name = region + '_dswrf' + delta[0]
    interp = interp_data(dswrf_dict[region].dswrf, delta[1])
    days_frame[column_name] = interp
      
days_frame = days_frame.copy()

add v wind

In [ ]:
f_name = working_directory + '/Data/VWND/vwnd.10m.gauss.'
vwnd_dict = get_regionalized_map_data(f_name, '.nc', 'vwnd', regions, year_start, year_end)
for region in vwnd_dict.keys():
  interp = interp_data(vwnd_dict[region].vwnd, days_frame['time'])
  column_name = region + '_vwnd'
  days_frame[column_name] = interp

  for delta in offset_dates:
    column_name = region + '_vwnd' + delta[0]
    interp = interp_data(vwnd_dict[region].vwnd, delta[1])
    days_frame[column_name] = interp

days_frame = days_frame.copy()

add uwnd

In [ ]:
f_name = working_directory + '/Data/UWND/uwnd.10m.gauss.'
uwnd_dict = get_regionalized_map_data(f_name, '.nc', 'uwnd', regions, year_start, year_end)
for region in uwnd_dict.keys():
  interp = interp_data(uwnd_dict[region].uwnd, days_frame['time'])
  column_name = region + '_uwnd'
  days_frame[column_name] = interp

  for delta in offset_dates:
    column_name = region + '_uwnd' + delta[0]
    interp = interp_data(uwnd_dict[region].uwnd, delta[1])
    days_frame[column_name] = interp

days_frame = days_frame.copy()

In [ ]:
#let's copy and normalize 
normalized_df = days_frame.copy()
feature_columns = normalized_df.columns.tolist()
if 'time' in feature_columns:
  feature_columns.remove('time')

print(feature_columns)

In [ ]:
for column in feature_columns:
  normalized_df[column] = (normalized_df[column] - normalized_df[column].min()) / (normalized_df[column].max() - normalized_df[column].min())

display(normalized_df)

In [ ]:
# save non-normalized
if days_frame.isnull().values.any():
  total_null = days_frame.isnull().sum().sum()
  print('Total null values: ', total_null)

days_frame.dropna(inplace=True)
pickle_name = working_directory + '/Data/Pickles/correlation_frame.pkl'
days_frame.to_pickle(pickle_name)

In [ ]:
# save normalized_df
if normalized_df.isnull().values.any():
  total_null = normalized_df.isnull.sum().sum()
  print('Total null values: ', total_null)

normalized_df.dropna(inplace=True)
pickle_name = working_directory + '/Data/Pickles/correlation_frame_normalized.pkl'
normalized_df.to_pickle(pickle_name)

Clean up and save pickles

In [ ]:
drive.flush_and_unmount()